# Herrington to Hart-Miller Island

## OpenCPN Planning Extract

This is the raw block of text to start with. We need to ingest the text to create a Route from it.

In [ ]:
openCPN = """
Hart-Miller
Name	Hart-Miller
Depart From	Hart-Muller Is
Destination	Herrington Harbour N
Total distance	 36.1 NMi
Speed (Kts)	6
Departure Time (%x %H:%M)	08/30/2021 09:14
Time enroute	 6H  1M

Leg	To waypoint	Distance	Bearing	Latitude	Longitude	ETE	ETA	Speed	Next tide event	Description	Course	ETD		
---	Hart-Milller Is	 30.3 NMi	28 °M	39° 15.4' N	076° 22.4' W	 5H  3M	Start: 08/30/2021 05:14 (Nighttime)	6			267 °M			
1	Hawk Cove G3	  1.0 NMi	267 °M	39° 15.2' N	076° 23.6' W	 9M 34S	08/30/2021 05:33 (MoTwilight)	6			192 °M			
2	Pleasure Is Ch N	  0.9 NMi	192 °M	39° 14.3' N	076° 23.7' W	 8M 57S	08/30/2021 05:42 (MoTwilight)	6			196 °M			
3	Pleasure Is Ch S	  0.9 NMi	196 °M	39° 13.4' N	076° 23.8' W	 9M  5S	08/30/2021 05:51 (MoTwilight)	6			171 °M			
4	Brewerton Channel	  4.3 NMi	171 °M	39° 09.4' N	076° 21.9' W	42M 31S	08/30/2021 06:34 (Sunrise)	6			196 °M			
5	Sandy Point Shoal	  8.5 NMi	196 °M	39° 00.9' N	076° 22.8' W	 1H 25M	08/30/2021 07:59 (Daytime)	6			209 °M			
6	Thomas Point	  7.7 NMi	209 °M	38° 53.6' N	076° 25.8' W	 1H 17M	08/30/2021 09:16 (Daytime)	6		CRTD 06-DEC-96 15:55	209 °M			
7	Chesapeake 85A	  3.8 NMi	209 °M	38° 50.0' N	076° 27.4' W	38M 29S	08/30/2021 09:54 (Daytime)	6			218 °M			
8	Herring Bay G1	  5.8 NMi	218 °M	38° 44.8' N	076° 30.7' W	58M  2S	08/30/2021 10:52 (Daytime)	6			258 °M			
9	Herring Bay	  1.4 NMi	258 °M	38° 44.2' N	076° 32.4' W	14M 28S	08/30/2021 11:07 (Daytime)	6			1 °M			
10	Herring Bay G3	  1.2 NMi	1 °M	38° 45.4' N	076° 32.7' W	11M 35S	08/30/2021 11:18 (Daytime)	6			345 °M			
11	HHN Entrance	  0.6 NMi	345 °M	38° 45.9' N	076° 33.0' W	 6M 14S	08/30/2021 11:24 (Daytime)	6			Arrived			
"""

In [ ]:
from navtools import opencpn_table
from io import StringIO

In [ ]:
file = StringIO(openCPN)
route = opencpn_table.route = opencpn_table.Route.load(file)

In [ ]:
route.title

In [ ]:
route.summary

## Confirm the source data

This includes the OpenCPN ETA, which is (usually) the date we copied the route.

Important. Leg 0 is misleading. It's how far OpenCPN thinks we need to go to get ready to start.
We must ignore it.

We can either check for ``leg.leg == 0`` or use a ``legs[1:]`` slice.

In [ ]:
for leg in route.legs[1:]:
    print(f"{leg.waypoint.name}, {leg.waypoint.lat:%02.0d° %6.3m}, {leg.waypoint.lon:%03.0d° %6.3m}, {leg.distance}, {leg.bearing}, {leg.speed}, {leg.ETE.h}h {leg.ETE.m:d}m, {leg.ETA}")

## Replace the departure time and compute ETA's

In [ ]:
from IPython.display import Markdown as md

In [ ]:
from navtools.analysis import parse_date

In [ ]:
ETD = parse_date("2021-9-17 8:00")
speed = 5.75

In [ ]:
%%capture route_2

print("|", " | ".join(["Name","Lat","Lon","Dist","Dir","ETE","ETA"]), "|")
print("|", " | ".join(["----","---","---","------:","---:","------","------------"]), "|")

ETA = ETD
for leg in route.legs[1:]:
    ETE = opencpn_table.Duration.fromfloat(hours=leg.distance / speed)
    ETA = ETA + ETE.timedelta
    print(f"| {leg.waypoint.name} | {leg.waypoint.lat:%02.0d° %06.3m} | {leg.waypoint.lon:%03.0d° %06.3m} | {leg.distance:0.1f} | {leg.bearing:03.0f} | {ETE.h}h {ETE.m:d}m | {ETA} |")

In [ ]:
md(route_2.stdout)

# Compute a Reciprocal Course

With, of course, a distinct ETD.

This is an analysis task, not an OpenCPN formatting task.

In [ ]:
from navtools import planning
from navtools.navigation import declination, Angle

In [ ]:
recip_waypoints = list(leg.waypoint for leg in reversed(route.legs))

In [ ]:
recip_waypoints

In [ ]:
ETD = parse_date("2021-9-15 8:00")
speed = 5.75

In [ ]:
recip_plan = list(planning.gen_schedule(recip_waypoints, declination, ETD, speed))

In [ ]:
%%capture route_1

print("|", " | ".join(["name","Lat","Lon","Distance","Bearing","ETE","ETA"]), "|")
print("|", " | ".join(["----","---","---","------:","---:","------","------------"]), "|")

ETA = ETD
for leg in recip_plan:
    ETE = opencpn_table.Duration.fromfloat(hours=leg.distance / speed)
    ETA = ETA + ETE.timedelta
    print(f"| {leg.waypoint.name} | {leg.waypoint.lat:%02.0d° %6.3m} | {leg.waypoint.lon:%03.0d° %6.3m} | {leg.distance:0.1f} | {leg.next_course or Angle.fromdegrees(0):%03.0d} | {ETE.h}h {ETE.m:d}m | {ETA} |")

In [ ]:
md(route_1.stdout)

# Final Float Plan

The float plan doc has four parts. 

1. Identification (Boilerplate)
2. Itinerary (Markdown, computed above)
3. Any notes or clarifications
4. USCG RCC Contact Information (Boilerplate)

There are several ways to publish a PDF from this.

1. Emit a markdown file composed of the various parts, and run a separate Markdown-to-PDF converter (i.e. PyMarkdown)

2. Use the Jupyter markdown capability by creating a second Notebook that contains *only* the final float-plan document. All the computation cells will have ``hide-input`` tag in their metadata to conceal the data imports.

In [ ]:
from pathlib import Path

In [ ]:
identification_path = Path("identification.md")
identification = identification_path.read_text()

In [ ]:
SAR = """
- RCC Norfolk. Commander 5th Coast Guard District Portsmouth, Virginia. Mid-Atlantic states including the majority of New Jersey down to the North Carolina/South Carolina Border. (757) 398-6231

- RCC Miami. Commander 7th Coast Guard District Miami, Florida. Southeast states from the South Carolina/North Carolina border around to the eastern end of the Florida panhandle plus a large portion of the Caribbean Sea. (305) 415-6800

- Bahamas Air-Sea Rescue Association (BASRA). Nassau, Bahamas. (242) 325-8864
"""

In [ ]:
md(identification)

In [ ]:
md(route_1.stdout)

In [ ]:
notes = None

In [ ]:
md(SAR)

## Option 1: Single .MD file

In [ ]:
from string import Template
import datetime

In [ ]:
fp_template = Template("""\
# Float Plan for S/V Red Ranger

${Now}

# Vessel Identification

${Identification}

# Itinerary

${Itinerary}

# Notes

${Notes}

# Search and Rescue Contacts
${SAR}
""")

In [ ]:
target = Path("To Hart-Miller.md")
body = fp_template.substitute(
    Now=datetime.datetime.now().isoformat(),
    Identification=identification,
    Itinerary=route_1.stdout,
    Notes=notes,
    SAR=SAR,
)
target.write_text(body)

When we create HTML, we need to "wrap" the HTML into a larger document. This can have a header that provides appropriate CSS styling that helps with printing or saving as PDF.

This works, but it limited to creating HTML only.

    !python -m markdown -x markdown.extensions.tables "To Hart-Miller.md" >"To Hart-Miller.html"

This is slightly more general, because it relies on the super-powerful pandoc tool.

In [ ]:
!pandoc --from markdown+pipe_tables "To Hart-Miller.md" -o "To Hart-Miller.html"

In [ ]:
!pandoc --from markdown+pipe_tables "To Hart-Miller.md" -o "To Hart-Miller.pdf"

In [ ]:
target = Path("From Hart-Miller.md")
body = fp_template.substitute(
    Now=datetime.datetime.now().isoformat(),
    Identification=identification,
    Itinerary=route_2.stdout,
    Notes=notes,
    SAR=SAR,
)
target.write_text(body)

In [ ]:
!pandoc --from markdown+pipe_tables "From Hart-Miller.md" -o "From Hart-Miller.pdf"

## Option 2: Save itineraries for a separate notebooks

In [ ]:
route_path1 = Path("to_hart_miller.md")
route_path1.write_text(route_1.stdout)

In [ ]:
route_path2 = Path("from_hart_miller.md")
route_path2.write_text(route_2.stdout)

Open the ``FloatPlanTemplate.ipynb`` notebook and customize to ingest the Markdown itinerary.

Then export the notebook as a PDF file using Jupyter Lab's internal pandoc to create the final PDF.

Or

     jupyter nbconvert FloatPlanTemplate.ipynb --to pdf --no-input